<a href="https://colab.research.google.com/github/sindla97/Recommender_systems/blob/main/content_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kagglehub

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dylanjcastillo/7k-books-with-metadata")

print("Path to dataset files:", path)

Path to dataset files: /Users/hemanth/.cache/kagglehub/datasets/dylanjcastillo/7k-books-with-metadata/versions/3


In [ ]:
path

'/Users/hemanth/.cache/kagglehub/datasets/dylanjcastillo/7k-books-with-metadata/versions/3'

In [ ]:
import pandas as pd
import numpy as np
df_books=pd.read_csv(path+'/books.csv')
df_books.head()

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count
0,9780002005883,0002005883,Gilead,NaN,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0
1,9780002261982,0002261987,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0
2,9780006163831,0006163831,The One Tree,NaN,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,1982.0,3.97,479.0,172.0
3,9780006178736,0006178731,Rage of angels,NaN,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0
4,9780006280897,0006280897,The Four Loves,NaN,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0


In [ ]:
df_books.loc[0,'thumbnail']

'http://books.google.com/books/content?id=KQZCPgAACAAJ&printsec=frontcover&img=1&zoom=1&source=gbs_api'

In [ ]:
df_books.shape

(6810, 12)

In [ ]:
df_books.isbn13.nunique()

6810

In [ ]:
df_books[['title','description']].iloc[0,1]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [ ]:
!pip install huggingface_hub

In [ ]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
model = AutoModel.from_pretrained("xlm-roberta-base")



In [ ]:
df_books=df_books[~df_books['description'].isnull()]

In [ ]:
import re

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)  # normalize whitespace
    text = re.sub(r'[^\x00-\x7F]+', '', text)  # remove non-ASCII (if needed)
    return text.strip()

df_books['cln_description']=df_books['description'].apply(clean_text)
df_books['cln_description'].head()

0    A NOVEL THAT READERS and critics have been eag...
1    A new 'Christie for Christmas' -- a full-lengt...
2    Volume Two of Stephen Donaldson's acclaimed se...
3    A memorable, mesmerizing heroine Jennifer -- b...
4    Lewis' work on the nature of love divides love...
Name: cln_description, dtype: object

In [ ]:
df_books.head(20)

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,cln_description
0,9780002005883,0002005883,Gilead,NaN,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,A NOVEL THAT READERS and critics have been eag...
1,9780002261982,0002261987,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,A new 'Christie for Christmas' -- a full-lengt...
2,9780006163831,0006163831,The One Tree,NaN,Stephen R. Donaldson,American fiction,http://books.google.com/books/content?id=OmQaw...,Volume Two of Stephen Donaldson's acclaimed se...,1982.0,3.97,479.0,172.0,Volume Two of Stephen Donaldson's acclaimed se...
3,9780006178736,0006178731,Rage of angels,NaN,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,"A memorable, mesmerizing heroine Jennifer -- b..."
4,9780006280897,0006280897,The Four Loves,NaN,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,Lewis' work on the nature of love divides love...
5,9780006280934,0006280935,The Problem of Pain,NaN,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,"""In The Problem of Pain, C.S. Lewis, one of th..."
6,9780006353287,0006353282,An Autobiography,NaN,Agatha Christie,"Authors, English",http://books.google.com/books/content?id=c49GQ...,Donation.,1977.0,4.27,560.0,3975.0,Donation.
7,9780006380832,0006380832,Empires of the Monsoon,A History of the Indian Ocean and Its Invaders,Richard Hall,"Africa, East",http://books.google.com/books/content?id=MuPEQ...,Until Vasco da Gama discovered the sea-route t...,1998.0,4.41,608.0,65.0,Until Vasco da Gama discovered the sea-route t...
8,9780006470229,000647022X,The Gap Into Madness,Chaos and Order,Stephen R. Donaldson,"Hyland, Morn (Fictitious character)",http://books.google.com/books/content?id=4oXav...,A new-cover reissue of the fourth book in the ...,1994.0,4.15,743.0,103.0,A new-cover reissue of the fourth book in the ...
9,9780006472612,0006472613,Master of the Game,NaN,Sidney Sheldon,Adventure stories,http://books.google.com/books/content?id=TkTYp...,Kate Blackwell is an enigma and one of the mos...,1982.0,4.11,489.0,43540.0,Kate Blackwell is an enigma and one of the mos...


In [ ]:
import torch

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Using device:", device)

Using device: mps


In [ ]:
import gc
import torch

# Delete unused variables
del model, encoded_input, outputs, outputs2  # or any large tensors
gc.collect()

# Clear MPS cache
torch.mps.empty_cache()

In [ ]:
model.to(device)
model.eval()

XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine

In [ ]:
# prepare input
text = df_books['cln_description'].to_list()
encoded_input = tokenizer(text, return_tensors='pt', max_length=512,truncation=True, padding=True, padding_side='right')

encoded_input[0]

Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [ ]:
encoded_input['input_ids'][0:100]

tensor([[     0,     62, 195488,  ...,      1,      1,      1],
        [     0,     62,   3525,  ...,      1,      1,      1],
        [     0,  98872,  32964,  ...,      1,      1,      1],
        ...,
        [     0,  19953,     53,  ...,      1,      1,      1],
        [     0,   6561,    111,  ...,      1,      1,      1],
        [     0,    581,   4734,  ...,      1,      1,      1]])

In [ ]:
encoded_input = {k: v.to(device) for k, v in encoded_input.items()}

In [ ]:
# outputs = model(**encoded_input)
len(text)/25

261.92

In [ ]:
# forward pass
outputs = []

batch_size = 25
for i in range(0, batch_size*262, batch_size):
    encoded_input2 = {k: v[i:i+batch_size].to(device) for k, v in encoded_input.items()}
    with torch.no_grad():
        outputs2 = model(**encoded_input2)
    outputs.append(outputs2.last_hidden_state[:, 0, :] )

embeddings = torch.cat(outputs, dim=0)

In [ ]:
embeddings.shape

torch.Size([6548, 768])

In [ ]:
cls_np = embeddings.cpu().numpy().astype("float32")

In [ ]:
cls_np

array([[ 0.57507765,  0.3550621 ,  0.5599906 , ..., -0.13909203,
         0.5413295 ,  0.03729209],
       [ 0.18777524, -0.08851093, -0.10397593, ..., -0.19964676,
        -0.47347963,  0.02475517],
       [-0.0753711 ,  0.12534392, -0.21612324, ..., -0.24536994,
         0.02377414, -0.29631615],
       ...,
       [ 0.4618827 ,  0.3611526 ,  0.10759313, ..., -0.51481545,
         0.1797769 ,  0.17410612],
       [ 0.4618827 ,  0.3611526 ,  0.10759313, ..., -0.51481545,
         0.1797769 ,  0.17410612],
       [ 0.4618827 ,  0.3611526 ,  0.10759313, ..., -0.51481545,
         0.1797769 ,  0.17410612]], dtype=float32)

In [ ]:
!pip install faiss-cpu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 27.8 MB/s eta 0:00:00


In [ ]:
import faiss
import pandas as pd



# Create FAISS index (cosine)
# Add vectors to index
cls_np_norm = cls_np / np.linalg.norm(cls_np, axis=1, keepdims=True)
index = faiss.IndexFlatIP(cls_np_norm.shape[1])  # IP = Inner Product ≈ cosine
index.add(cls_np_norm)

# Save text metadata
# text_df = df_books
df_books.to_csv("/users/hemanth/Documents/data_science/data/books_cbrecomendation.csv", index=False)
faiss.write_index(index, "/users/hemanth/Documents/data_science/data/cls_index.faiss")

In [ ]:
def topk_recomendations(text_book_input,model,k, description=''):
    if len(description.split())> 10:
        query_input = tokenizer(clean_text(description), return_tensors="pt", truncation=True, padding=True,padding_side='right').to(device)
        with torch.no_grad():
            query_output = model(**query_input)
        query_embedding = query_output.last_hidden_state[:, 0, :].cpu().numpy().astype("float32")
        distances, indices = index.search(query_embedding, k)
        return distances, indices
    elif len(text_book_input)>0:
        query_df =df_books[df_books['title'].str.contains(text_book_input)]
        if query_df.shape[0]==0:
            print('text book name out-of-scope: provide/ try with short description')
        else:
            final_indices={}
            for i in query_df.index.tolist():
                # print(query_df.loc[i,'cln_description'], query_df.loc[i,'title'])
                query_input = tokenizer(query_df.loc[i,'cln_description'], return_tensors="pt", truncation=True, padding=True,padding_side='right').to(device)
                with torch.no_grad():
                    query_output = model(**query_input)
                query_embedding = query_output.last_hidden_state[:, 0, :].cpu().numpy().astype("float32")
                distances, indices = index.search(query_embedding, k)
                # print(distances, indices)
                if not final_indices:
                    final_indices=dict(zip(indices[0].tolist(), distances[0].tolist()))
                else:
                    for i in range(len(indices[0])):
                        if indices[0][i] in final_indices.keys():
                            # print(indices[0][i],distances[i],final_indices[indices[0][i]])
                            final_indices[indices[0][i]]=min(distances[0][i],final_indices[indices[0][i]])
                        else:
                            final_indices[indices[0][i]]=distances[0][i]
            sorted_dict = dict(sorted(final_indices.items(), key=lambda item: item[1]))
            return  np.array(list(sorted_dict.values())[:k]), np.array(list(sorted_dict.keys())[:k])




In [ ]:
topk_recomendations('Gilead',model=model,k=5)

[[18.58976  18.589441 18.58941  18.589037 18.58801 ]] [[5632 3072 4096    0 6144]]


(array([18.58801079, 18.58903694, 18.58941078, 18.5894413 , 18.58975983]),
 array([6144,    0, 4096, 3072, 5632]))

In [ ]:
k = 5  # top-k results
distances, indices = topk_recomendations("Darkness",model=model,k=5)
indices
# Show results
for i, idx in enumerate(indices):
    print(f"Rank {i+1}: {df_books.iloc[idx]['title']} (Distance: {distances[i]:.4f})")

Gareth is just a frightened young lad when he is drafted in to be the whipping-boy for the unruly Prince Dagnarus. Yet as they grow to be men, an unshakable bond is formed. And it is their destiny for them to become the two most powerful - and feared - men in the kingdom. But when Dagnarus decides to become a knight himself, he begins a chain of evil events that threaten to destroy the entire kingdom. Only by uniting the Sovereign Stone can peace be restored, and it is up to Dagnarus' brother Helmos to try to achieve this impossible task, using his father's soldiers against his brother's army. All the while, Gareth is gaining mastery over the sinister Void magic, and he is devoted to helping his childhood companion.. This gripping story of magical power and corruption takes the unique viewpoint of focusing on the evil characters, as we follow them from boys to men. Filled with action, suspense and wonderfully imaginative characters, Water From the Well of Darkness is set to elevate the